In [75]:
import typing
from agents import Agent, trace, Runner, function_tool, WebSearchTool, gen_trace_id, ItemHelpers
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import requests
import os
from typing import List, Dict, Any
from IPython.display import display, Markdown
from openai.types.responses import ResponseTextDeltaEvent

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [2]:
model = "gpt-4o-mini"

In [ ]:
SEARCH_INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search Agent",
    instructions=SEARCH_INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model=model,
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
#Test search agent
message = "Latest AI Agent frameworks in July 2025"

with trace(search_agent):
    result = await Runner.run(search_agent, message)
    

display(Markdown(result.final_output))

In [ ]:
HOW_MANY_SEARCHES = 3
PLANNER_INSTRUCTIONS = f"You are a helpful research assistant.  Given a query, please come up with a set web searches terms \
    to perform to best answer the query.  Output  {HOW_MANY_SEARCHES} terms to search for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=PLANNER_INSTRUCTIONS,
    model=model,
    output_type=WebSearchPlan,
)

In [32]:
message = "Invest in Apple stock now 07/31/2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)
    print(f"Will perform {len(result.final_output.searches)} searches")


searches=[WebSearchItem(reason='To understand the current performance and projections for Apple stock in 2025.', query='Apple stock price analysis July 2025'), WebSearchItem(reason="To gather expert opinions and market predictions regarding Apple's future performance.", query='Apple stock investment outlook 2025'), WebSearchItem(reason="To review recent news and developments that could impact Apple's stock price.", query='Apple company news July 2025')]
Will perform 3 searches


In [15]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with given subjecy and HTML body"""
    return requests.post(
  		"https://api.mailgun.net/v3/sandbox979b287399f848ed9122f9a12c836b17.mailgun.org/messages",
  		auth=("api", os.getenv('MAIL_GUN_API_KEY', 'MAIL_GUN_KEY')),
  		data={"from": "Mailgun Sandbox <postmaster@sandbox979b287399f848ed9122f9a12c836b17.mailgun.org>",
			"to": "Son M Ngo <sonmngo@gmail.com>",
  			"subject": subject,
  			"html": html_body})

In [16]:
send_email

FunctionTool(name='send_email', description='Send out an email with given subjecy and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002C22B27B9C0>, strict_json_schema=True, is_enabled=True)

In [ ]:
EMAIL_INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=EMAIL_INSTRUCTIONS,
    tools=[send_email],
    model=model,
)


In [ ]:
WRITER_INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

    references: list[str] = Field(description="List of name and url of the references used in the report")


writer_agent = Agent(
    name="WriterAgent",
    instructions=WRITER_INSTRUCTIONS,
    model=model,
    output_type=ReportData,
)

In [19]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [20]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [ ]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")



In [39]:
# Create agent to perform the searches
search_agent_tool = search_agent.as_tool(tool_name="WebSearchTool", tool_description="Perform a single web search based on a query and a reason.")
perform_searches_agent = Agent(
    name="BatchSearchAgent",
    instructions=(
        "You are a search orchestrator. You receive a WebSearchPlan containing multiple search queries. "
        "For each item, call WebSearchTool to perform the search. "
        "Return a list of all results."
    ),
    model=model,
    tools=[search_agent_tool]
)

In [40]:
print(perform_searches_agent)

Agent(name='BatchSearchAgent', instructions='You are a search orchestrator. You receive a WebSearchPlan containing multiple search queries. For each item, call WebSearchTool to perform the search. Return a list of all results.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[FunctionTool(name='WebSearchTool', description='Perform a single web search based on a query and a reason.', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'WebSearchTool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<loca

In [50]:
planner_agent_tool = planner_agent.as_tool(tool_name="SearchPlanTool", tool_description="Plan which searches to perform for a given query")
perform_searches_tool = perform_searches_agent.as_tool(
    tool_name="BatchWebSearchTool",
    tool_description="Perform all searches in a WebSearchPlan by calling WebSearchTool for each query."
)

writer_agent_tool = writer_agent.as_tool(
    tool_name="ReportWriterTool",
    tool_description="Write a summarized report based on search results."
)

email_agent_tool = email_agent.as_tool(
    tool_name="EmailSenderTool",
    tool_description="Send the generated report via email."
)


In [48]:
CONTROLLER_INSTRUCTIONS = f"You conduct research for a given query. \n" \
"1. Use SearchPlanTool to plan searches. \n" \
"2. Use BatchWebSearchTool to execute all searches. \n" \
"3. Summarize the results with ReportWriterTool. \n" \
"4. Send the final report using EmailSenderTool."

controller_agent = Agent(
    name="ResearchCoordinator",
    instructions=CONTROLLER_INSTRUCTIONS,
    model=model,
    tools=[
        planner_agent_tool,
        perform_searches_tool,
        writer_agent_tool,
        email_agent_tool
    ]
)

In [52]:
query = "Latest AI Agent frameworks in 2025"
with trace("Research Trace"):
    result = await Runner.run(controller_agent, query)
    print("Hooray!  I am done.")

Hooray!  I am done.


In [ ]:
#Now running with stream to print out intermediate states
TOOL_LABELS = {
    "SearchPlanTool": "Planning searches",
    "BatchWebSearchTool": f"Performing {HOW_MANY_SEARCHES} web searches",
    "ReportWriterTool": "Writing the report",
    "EmailSenderTool": "Sending the report via email"
}


query = "Latest AI Agent frameworks in 2025"


with trace("Research Trace"):
    stream =  Runner.run_streamed(controller_agent, query)

    async for event in stream.stream_events():
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                tool_name = event.item.raw_item.name
                #tool_args = event.item.raw_item.arguments
                #call_id = event.item.raw_item.call_id
                
                print(f"\n {TOOL_LABELS.get(tool_name, tool_name)}")
                #print(f"\n   Arguments: {tool_args}")
                #print(f" \n  Call ID: {call_id}")
            elif event.item.type == "tool_call_output_item":
                #print(f"-- Tool output: {event.item.output}")
                continue
            elif event.item.type == "message_output_item":
                #print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
                continue
            else:
                pass  # Ignore other event types

    print("n\nFinal Output:")
    print(stream.final_output)
    print("=== Run complete ===")

    print("Hooray!  I am done.")

In [ ]:


async def run_research_agent(query, agent):
    """
    Run the research agent with streaming output
    
    Args:
        query (str): The research query
        agent: The controller agent to use
    
    Returns:
        The final output from the stream
    """
    stream = Runner.run_streamed(agent, query)

    async for event in stream.stream_events():
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                tool_name = event.item.raw_item.name
                print(f"\n {TOOL_LABELS.get(tool_name, tool_name)}")
                
            elif event.item.type == "tool_call_output_item":
                continue
            elif event.item.type == "message_output_item":
                continue
            else:
                pass  # Ignore other event types

    print("\n\nFinal Output:")
    #print(stream.final_output)
    print("=== Run complete ===")
    print("Hooray! I am done.")
    
    return stream.final_output



In [ ]:
query = "Latest AI Agent frameworks in 2025"
with trace("Research Trace"):
    result = await run_research_agent(query, controller_agent)
    print("Final Result")
    print(result)

In [83]:
def run(query: str):
    print(f"query is {query}")
    return "testing with query: " + query

In [ ]:
import gradio as gr
with gr.Blocks(theme=gr.themes.Default(primary_hue="sky")) as ui:
    gr.Markdown("# Deep Research")
    query_textbox = gr.Textbox(label="What topic would you like to research?")

    with gr.Row():
        run_button = gr.Button("Run", variant="primary")
        clear_button = gr.Button("Clear", variant="secondary")

    report = gr.Markdown(label="Report")
    
    run_button.click(fn=run, inputs=query_textbox, outputs=report)

    clear_button.click(fn=lambda: ("", ""), inputs=None, outputs=[query_textbox, report])

    query_textbox.submit(fn=run, inputs=query_textbox, outputs=report)

print("Launching Gradio interface...")
ui.launch(inbrowser=True)